<a href="https://colab.research.google.com/github/mohammed-el-barhichi/Modeling-Crowd-Behaviour/blob/main/Modeling_Crowd_Behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from sklearn.linear_model import LinearRegression
import pickle
from matplotlib.patches import Rectangle

#initialisation
nombrePietons=20 ; largeurPiece=5 ; longeurPiece=5 ; k=1.2*(10**3)
h=10**(-2) #pas du temps
T=0.5 #temps de relaxation
listePortes=[[largeurPiece,longeurPiece/2]] #en cas d'existance de plusieurs portes
Vn=np.zeros((nombrePietons,2)) #vitesse initiale


def initialisation():
    global coordonnees,vitesseSouhaitee,masse,rayon

    #génération des coordonnées aléatoires des piétons
    coordonnees=np.zeros((nombrePietons,2))
    for i in range(nombrePietons):
        coordonnees[i,0]=(random.uniform(0, largeurPiece))
        coordonnees[i,1]=(random.uniform(0, largeurPiece))

    #génération des vitesses souhaitées aléatoires entre 1.1m/s et 1.48 m/s
    vitesseSouhaitee=np.zeros(nombrePietons)
    for i in range(nombrePietons):
        vitesseSouhaitee[i]=random.uniform(1.1,1.48)

    #génération des masses aléatoires entre 50kg et 100kg
    masse=np.random.randint(low=50, high=100, size=nombrePietons)

    #choix des rayons convenables des pietons entre 0.2m et 0.25m
    rayon=np.array( [ 0.001*masse[i]+0.15 for i in range(nombrePietons) ] )

initialisation()

#listes des couleurs choisi pour représenter chaque piéton
t='tab:'
colors = ['b','g','r','c','m','y',t+'blue',t+'orange','limegreen','orangered',t+'purple',t+'brown',t+'pink',t+'grey','aquamarine','lime','yellow','pink','indigo','gold']


#traçage des données initiales
def environnement():
    fig, ax = plt.subplots()
    ax.plot()
    plt.axis('square')
    plt.xlim([0, largeurPiece]); plt.ylim([0, longeurPiece])
    plt.xlabel('Coordonnées en x') ; plt.ylabel('Coordonnées en y')
    plt.scatter(listePortes[0][1]+0.05,listePortes[0][1],color= 'pink',marker='s',edgecolors='r',s=400, label='Porte')
    plt.text(listePortes[0][0],listePortes[0][1],"Porte")

environnement()
for N in range(nombrePietons):
    plt.scatter(coordonnees[N,0],coordonnees[N,1], color=colors[N],s=rayon[N]*200)
    plt.text(coordonnees[N,0],coordonnees[N,1], str(N+1))


#définition des fonctions:

#calcul de la distance entre le piéton et la porte
def distanceToDoor(numeroPieton,coordsPorte):
	return math.sqrt( (coordsPorte[0]-coordonnees[numeroPieton][0])**2 +(coordsPorte[1]-coordonnees[numeroPieton][1])**2 )

#détermination de la porte la plus proche
def closestDoor(numeroPieton,listePorte):
	distances = []
	for p in range(0,len(listePorte)):
		distances.append(distanceToDoor(numeroPieton,listePorte[p]))
	return distances.index(min(distances))

#calcul de la direction souhaitée d'un pieton (vers la plus proche porte)
def direction(numeroPieton):
	porte=closestDoor(numeroPieton,listePortes)
	X=listePortes[porte][0]- coordonnees[numeroPieton][0]
	Y=listePortes[porte][1] -coordonnees[numeroPieton][1]
	return np.array([X/distanceToDoor(numeroPieton,listePortes[porte]),Y/distanceToDoor(numeroPieton,listePortes[porte])])

#calcul de la force motrice F
def calculF(numeroPieton,Vn):
	dir=direction(numeroPieton)
	return (masse[numeroPieton]*(vitesseSouhaitee[numeroPieton]*dir-Vn[numeroPieton]))/T

#calcul de la distance entre deux piétons
def distanceEntre(i,j,coordonnees):
	return math.sqrt((coordonnees[i][0]-coordonnees[j][0])**2+(coordonnees[i][1]-coordonnees[j][1])**2)

#détection des piétons en contact avec un piéton
def detectionContact(i,coordonnees):
    pietonsTouches=[]
    for j in range(0,nombrePietons): #j pour les autres piétons
        if j != i :
            Dij = distanceEntre(i,j,coordonnees) - (rayon[i]+rayon[j])
            if Dij <= 0 :
                pietonsTouches.append(j)
    return pietonsTouches

#calcul de la force pieton-pieton G
def calculG(numeroPieton,coordonnees):
	pietonsTouches=detectionContact(numeroPieton,coordonnees)
	G=np.zeros(2)
	for j in pietonsTouches:
		Dij = distanceEntre(numeroPieton,j,coordonnees) - (rayon[numeroPieton]+rayon[j])
		G=G+k*Dij*((coordonnees[j]-coordonnees[numeroPieton])/distanceEntre(j,numeroPieton,coordonnees))
	return G

#calcul de la force pieton-mur de la porte K
def calculK(numeroPieton,coordonnees):
	porte=closestDoor(numeroPieton,listePortes)
	K=np.zeros(2)
	if coordonnees[numeroPieton,1] < listePortes[porte][1]-0.2 or coordonnees[numeroPieton,1] > listePortes[porte][1]+0.2:
		D=largeurPiece-coordonnees[numeroPieton,0]-rayon[numeroPieton]
		if D <= 0 :
			K[0]=K[0]+k*D
	return K

#calcul de la vitesse d'un piéton à tout instant
def calculVitesse(numeroPieton,Vn,coordonnees):
    return Vn[numeroPieton]+(h/masse[numeroPieton]) * (calculF(numeroPieton,Vn)+calculG(numeroPieton,coordonnees)+calculK(numeroPieton,coordonnees))

#Calcul de la position d'un piéton à tout instant
def calculPosition (numeroPieton,Vn, coordonnees):
	return coordonnees[numeroPieton] + h*Vn[numeroPieton]


#traçage des trajets des piétons dasn une salle vide (figures 6,7,8,9 et 10 en ajoutant la force K)
environnement()
for N in range(nombrePietons):
    plt.text(coordonnees[N,0],coordonnees[N,1], str(N+1))

i=0
while min(coordonnees[:,0])<listePortes[0][0]: #pour s'assurer que tous les piétons ont évacués
    for N in range(0,nombrePietons):
        if coordonnees[N,0]<listePortes[0][0]:
            plt.scatter(coordonnees[N,0],coordonnees[N,1],facecolors='none', edgecolors=colors[N],s=20)
            Vn[N]=calculVitesse(N,Vn,coordonnees)
            coordonnees[N]=calculPosition(N,Vn,coordonnees)
        else:
            coordonnees[N]=np.array([1000,1000]) #pour éviter de prendre en compte les intéractions des piétons qui sont sortis
    i=i+1  #pour determiner le temps d'évacuation: on multiplie i final par h le pas du mouvement.
    if i == 25:
        temp=pickle.dumps(ax)
        plt.show() 
        ax=pickle.loads(temp)
    if i == 50:
        break
plt.show() 


#traçage des résultats de 50 simulations et la reggression linéaire (figure 11)

initialisation()
fig, ax = plt.subplots()
plt.xlim([0, 8]); plt.ylim([0, 20])
plt.yticks(np.arange(0,21,2))
plt.xlabel('Temps (s)') ; plt.ylabel('Nombre des piétons évacués')

#traçage des 50 simulations
X,Y=[],[]
for j in range(50):
    initialisation()
    i=0 ; p=0 ; Temps=[] ; Nombre=[]
    while min(coordonnees[:,0])<listePortes[0][0]:
        for N in range(0,nombrePietons):
            if coordonnees[N,0]<listePortes[0][0]:
                Vn[N]=calculVitesse(N,Vn,coordonnees)
                coordonnees[N]=calculPosition(N,Vn,coordonnees)
            else:
                if coordonnees[N][0] != 1000 : 
                    p+=1
                    Temps.append(i*h)
                    Nombre.append(p+1)
                    X.append(i*h)
                    Y.append(p+1)
                coordonnees[N]=np.array([1000,1000]) 
        i+=1
    Temps.append(i*h)
    Nombre.append(p+1)
    X.append(i*h)
    Y.append(p+1)
    if j == 0 :
        plt.plot(Temps,Nombre,color='cyan',label='50 simulations')
    else :
        plt.plot(Temps,Nombre,color='cyan')

#traçage de la régression linéaire
regr=LinearRegression()
X=np.array(X).reshape((-1,1))
Y=np.array(Y)
regr.fit(X,Y)
pente=regr.coef_
print(pente) #le coefficient de la pente de la droite
Z=regr.predict(X)
plt.plot(X,Z,color='b',label='Réression linéaire')
plt.legend()
plt.show()


#traçae de la progression des étudiants dans la salle d'une classe (figures 12,13)

coord1=[] 
for i in np.arange(0,6.1,1.2):
	coord1=coord1+[(0.7+j,1.8+i)for j in np.arange(0,5.9,1.4)]

#coordonnées des objets dans la classe
coordObjets=np.zeros((nombrePietons,2))
for i in range(nombrePietons):
    coordObjets[i,0]=coord1[i][0]
    coordObjets[i,1]=coord1[i][1]

#coordonnées initiales des etudiants dans la classe
coordonnees=np.zeros((nombrePietons,2))
for i in range(nombrePietons):
    coordonnees[i,0]=coord1[i][0]
    coordonnees[i,1]=coord1[i][1]+0.45

#placement des objets (les tables)
L=[] #liste des coordonnees des objets
for i in np.arange(0,6.1,1.2):
	L=L+[(0.45+j,1.625+i)for j in np.arange(0,5.9,1.4)]

def environnement():
    ax.plot()
    plt.axis('square')
    plt.xlim([0, largeurPiece]); plt.ylim([0, longeurPiece])
    plt.xlabel('Coordonnées en x') ; plt.ylabel('Coordonnées en y')
    for i in L:
        ax.add_patch(Rectangle(i,0.5,0.35))
    plt.scatter(listePortes[0][0]+0.1,listePortes[0][1],color= 'pink',marker='s',edgecolors='r',s=400, label='Porte')
    plt.text(listePortes[0][0],listePortes[0][1],"Porte")

#définition des fonctions

#calcul de la distance entre le piéton et l'objet
def distanceEntreObj(i,j,coordonnees,coordObjets):
	return math.sqrt((coordonnees[i][0]-coordObjets[j][0])**2+(coordonnees[i][1]-coordObjets[j][1])**2)

#détection des objets en contact avec un piéton
def detectionContactObjets(i,coordonnees,coordObjets):
    objetsTouches=[]
    for j in range(0,nombrePietons): #j pour les objets (car nombrePietons=nombreObjets)
            Dij = distanceEntreObj(i,j,coordonnees,coordObjets) - 2*rayon[i]
            if Dij <= 0 :
                objetsTouches.append(j)
    return objetsTouches

#calcul de la force pieton-objet L
def calculL(numeroPieton,coordonnees,coordObjets):
    objetsTouches=detectionContactObjets(numeroPieton,coordonnees,coordObjets)
    L=np.zeros(2)
    for j in objetsTouches:
        Dij = distanceEntreObj(numeroPieton,j,coordonnees,coordObjets) - 2*rayon[numeroPieton]
        L=L+k*Dij*((coordObjets[j]-coordonnees[numeroPieton])/distanceEntreObj(numeroPieton,j,coordonnees,coordObjets))
    return L

#calcul de la vitesse d'un piéton à tout instant (en ajoutant la force L)
def calculVitesse(numPieton,Vn,coordonnees):
    return Vn[numPieton]+(h/masse[numPieton])*(calculF(numPieton,Vn)+calculG(numPieton,coordonnees)+calculK(numPieton,coordonnees)+calculL(numPieton,coordonnees,coordObjets))


#traçage des positions des étudiants
def tracage():
    for N in range(0,nombrePietons):
        plt.scatter(coordonnees[N,0],coordonnees[N,1],facecolors='none',edgecolors='r',s=rayon[N]*200)
        plt.text(coordonnees[N][0],coordonnees[N][1], str(N+1))
    
fig, ax = plt.subplots()
environnement()
tracage() #t=0s

i=0
while min(coordonnees[:,0])<listePortes[0][0]:
    for N in range(0,nombrePietons):
        if coordonnees[N,0]<listePortes[0][0]:
            Vn[N]=calculVitesse(N,Vn,coordonnees)
            coordonnees[N]=calculPosition(N,Vn,coordonnees)
        else:
            coordonnees[N]=np.array([1000,1000])
    i=i+1  
    if i == 200:
        temp=pickle.dumps(ax)
        tracage() #t=i*h donc t=2s
        plt.show()
        ax=pickle.loads(temp)
    if i == 500:
        temp=pickle.dumps(ax)
        tracage() #t=5s
        plt.show()
        ax=pickle.loads(temp)
    if i==1000 :
        break
tracage() #t=10s
plt.show()


#traçage de l'effet 'Slower is faster' (figure 14)
nombrePietons=200 ; largeurPiece=20 ; longeurPiece=20 
environnement()
initialisation()
tracage() #t=0s
p=0 #compteur des pietons qui ont sorti
i=0
while min(coordonnees[:,0])<listePortes[0][0]:
    for N in range(0,nombrePietons):
        if coordonnees[N,0]<listePortes[0][0]:
            Vn[N]=calculVitesse(N,Vn,coordonnees)
            coordonnees[N]=calculPosition(N,Vn,coordonnees)
        else:
            if coordonnees[N][0] != 1000 : p+=1 #p nombre des piétons évacués
            coordonnees[N]=np.array([1000,1000])
    i=i+1
    if i == 800:
        temp=pickle.dumps(ax)
        tracage() #t=8s
        plt.show()
        ax=pickle.loads(temp)
    if i == 1600:
        break
tracage() #t=16s
print("le nombre des piétons évacués est :",p)
plt.show()